# Simple Regression NN

In [228]:
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import  DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from testTrainData import TrainData, TestData

# Model imports
from multiLabelRegression import MultiLabelRegression
from multiLayerMultiLabelRegression import MultiLayerMultiLabelRegression
from pytorch_forecasting.metrics import QuantileLoss, MultiLoss


In [ ]:
# Import modules from parent directory
sys.path.insert(0,'..')

from ipynb.fs.full.training_preprocessing import GetDataset

In [ ]:
df = GetDataset()

In [ ]:
# set x equal to all columns except for the transaction_count and workforce_type_1
x = df.drop(['transaction_count', 'workforce_type_1','workforce_type_2', 'workforce_type_3','workforce_type_4'], axis=1)
# convert x to a 2d array
x = x.values.tolist()


# set y equal to the transaction_count and workforce_type_1 columns
y = df[['transaction_count', 'workforce_type_1', 'workforce_type_2', 'workforce_type_3','workforce_type_4']]
# convert y to an array
y = y.values.tolist()

In [ ]:
TEST_SIZE = 0.33

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE)

In [ ]:
train_data = TrainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train))

test_data = TestData(torch.FloatTensor(X_test))

In [ ]:
def train(model, scheduler, criterion, num_epochs, train_loader):
    lowest_loss = 1000

    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            # Forward pass
            outputs = model(inputs)
            # Compute loss
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            scheduler.optimizer.zero_grad()
            loss.backward()
            scheduler.optimizer.step()

        scheduler.step(loss)
        print(scheduler.optimizer.param_groups[0]['lr'])

        # check if the current model has the lowest loss
        if loss < lowest_loss:
            # update the lowest loss variable
            lowest_loss = loss
            
            # save the current model checkpoint
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': scheduler.optimizer.state_dict(),
                'loss': lowest_loss
            }
            torch.save(checkpoint, 'saved/bestModelChkpt.pt')
            torch.save(model.state_dict(), 'saved/bestModel.pt')
            
        # Print progress
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Hyperparameters
LEARNING_RATE = 0.001
EPOCHS = 10000

In [ ]:
# Models
#model = MultiLayerMultiLabelRegression(11, 5)
model = MultiLabelRegression(11, 5)
model.to(device)

In [241]:
# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-8)


In [242]:
train(model, scheduler, criterion, EPOCHS, train_data)

0.001
Epoch [1/10000], Loss: 51.3826


KeyboardInterrupt: 

In [ ]:
# Save the torch model
# Load the best model
#model = MultiLabelRegression(11, 5)
model = MultiLayerMultiLabelRegression(11,5)

modelChkpt = torch.load('saved/singleLayerRegression1000epochChkpt.pt')
model.load_state_dict(torch.load('saved/multi_layer_linear_regression.pt'))
model.to(device)

In [ ]:
# Get the model loss
model_loss = modelChkpt['loss']
model_loss

## Evaluating model performance

In [ ]:
Xtest = torch.FloatTensor(X_test)
test_loader = DataLoader(dataset=Xtest, batch_size=1)

# Set the model to eval mode and generate a list of predictions for the test data
model.eval()
y_pred_list = []
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_pred_list.append(y_test_pred.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_pred_list

In [ ]:
# create an array of only the first object of each array inside y_pred_list
transaction_count_predictions = [a[0] for a in y_pred_list]
transaction_count_actuals = [a[0] for a in y_test]

In [ ]:
# Calculate the MSE and MAE between the predictions and the actual values
mse = mean_squared_error(y_test, y_pred_list)
mae = mean_absolute_error(y_test, y_pred_list)

print(f'MSE: {mse:.2f}, MAE: {mae:.2f}')

In [ ]:
# Calculate the MSE and MAE between only the transaction_count prediction and the actual values
mse = mean_squared_error(transaction_count_actuals[:128], transaction_count_predictions[:128])
mae = mean_absolute_error(transaction_count_actuals[:128], transaction_count_predictions[:128])

print(f'MSE: {mse:.2f}, MAE: {mae:.2f}')

In [ ]:
# Measuring against the baseline (63 samples)
mse = mean_squared_error(transaction_count_actuals[:63], transaction_count_predictions[:63])
mae = mean_absolute_error(transaction_count_actuals[:63], transaction_count_predictions[:63])

print(f'MSE: {mse:.2f}, MAE: {mae:.2f}')

In [ ]:
dist = torch.dist(torch.FloatTensor(transaction_count_predictions), torch.FloatTensor(transaction_count_actuals))

dist

In [ ]:
# Calculate SMAPE between actual and predicted values.
def smape(actual, predicted):
    actual = np.array(actual)
    predicted = np.array(predicted)
    smape_val = (100.0 / actual.size) * np.sum(2.0 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))
    return smape_val

In [ ]:
smape = smape(y_pred_list[:63], y_test[:63])
print(smape)

In [ ]:
# Graph the predictions vs the actual values
import matplotlib.pyplot as plt

#plt.scatter(transaction_count_predictions, transaction_count_actuals, alpha=0.5)
plt.plot(transaction_count_actuals[200:600], label='Actual')
plt.plot(transaction_count_predictions[200:600], label='Predicted')
plt.xlabel('Data Point Index')
plt.ylabel('Value')
plt.title('Predictions vs Actual Values')
plt.legend()
plt.show()